In [ ]:
import sys
sys.path.append('..')
from pcutils.core.config import load_config
from pcutils.core.workspace import Workspace
from pcutils.plot.histogram import Histogrammer
from pcutils.core.target import Target
from pcutils.core.constants import AMU_2_MEV
import matplotlib.pyplot as plt
import polars as pl
import numpy as np
from scipy import constants

QBRHO_2_P: float = 1.0e-9 * constants.speed_of_light * 100.0 * 10.0 #T * m -> MeV/c

In [ ]:
config = load_config('../local_config.json')
ws = Workspace(config.workspace)
nuclear_map = ws.get_nuclear_map()
target_material = Target(config.solver.gas_data_path, nuclear_map)
ejectile = nuclear_map.get_data(1, 2)
projectile = nuclear_map.get_data(6, 16)
target = nuclear_map.get_data(1, 2)
residual = nuclear_map.get_data(6, 16)
mass_amu = projectile.mass / AMU_2_MEV
proj_energy_start = 11.6 * mass_amu #MeV


In [ ]:

grammer = Histogrammer()
grammer.add_hist2d('ke_theta', (180, 1600), ((0.0, 90.0), (0.0, 80.0)))
grammer.add_hist2d('ke_phi', (360, 1600), ((0.0, 360.0), (0.0, 80.0)))
grammer.add_hist1d('ex', 750, (-5.0, 10.0))

In [ ]:
targ_vec = np.array([0., 0., 0., target.mass])
Q_mass = target.mass + projectile.mass - ejectile.mass - residual.mass
for run in range(config.run.run_min, config.run.run_max+1):
    df = None
    try:
        df = pl.read_parquet(ws.get_physics_file_path_parquet(run, ejectile))
    except Exception:
        continue

    vertices = df.select(['vertex_x', 'vertex_y', 'vertex_z']).to_numpy()
    distances = np.linalg.norm(vertices, axis=1)
    proj_energy = proj_energy_start - target_material.get_energy_loss(projectile, proj_energy_start, distances)

    brho = df.select('brho').to_numpy().flatten()
    momentum = df.select('brho').to_numpy().flatten() * float(ejectile.Z) * QBRHO_2_P
    kinetic_energy = np.sqrt(momentum**2.0 + ejectile.mass**2.0) - ejectile.mass
    polar = df.select('polar').to_numpy().flatten()
    az = df.select('azimuthal').to_numpy().flatten()

    qterm = 2.0/residual.mass * np.sqrt(projectile.mass * ejectile.mass * kinetic_energy * proj_energy) * np.cos(polar)
    Q_rxn = -1.0 * (kinetic_energy * (1.0 + ejectile.mass/residual.mass) + proj_energy * ( projectile.mass/residual.mass - 1.0) - qterm)

    grammer.fill_hist2d('ke_theta', np.rad2deg(polar), kinetic_energy)
    grammer.fill_hist2d('ke_theta_resid', np.rad2deg(polar), kinetic_energy)
    grammer.fill_hist2d('ke_phi', np.rad2deg(az), kinetic_energy)
    grammer.fill_hist1d('ex', Q_rxn + Q_mass)

In [ ]:
angle_range = np.linspace(0., np.pi*0.5, 1000)

ex = 0.0

Q = projectile.mass + target.mass - (ejectile.mass + residual.mass + ex)
term1 = np.sqrt(projectile.mass * ejectile.mass * proj_energy_start) / (ejectile.mass + residual.mass) * np.cos(angle_range)
term2 = (proj_energy_start * (residual.mass - projectile.mass) + residual.mass*Q) / (residual.mass + ejectile.mass)
eject_energy = term1 + np.sqrt(term1*term1 + term2)
eject_energy = eject_energy**2.0

In [ ]:
fig, ax = plt.subplots(1,2)
fig.set_figwidth(16)
fig.set_figheight(10)
mesh = grammer.draw_hist2d('ke_theta', ax[0], log_z=False)
ax[0].scatter(angle_range*180.0/np.pi, eject_energy, s=3, color='r')
ax[0].set_xlabel(r'$\theta_{Lab}$')
ax[0].set_ylabel('Kinetic Energy (MeV)')
ax[0].set_ylim(0.0, 80.0)
plt.colorbar(mesh, ax=ax[0])
mesh2 = grammer.draw_hist2d('ke_phi', ax[1], log_z=False)
ax[1].set_xlabel(r'$\phi_{Lab}$')
ax[1].set_ylabel('Kinetic Energy (MeV)')
plt.colorbar(mesh, ax=ax[1])
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,1)
fig.set_figwidth(16)
fig.set_figheight(10)
grammer.draw_hist1d('ex', ax)